In [9]:
import pandas as pd

# Tratando dados físicos e meteorológicos

In [10]:
pf_df = pd.read_csv("clima.csv", sep=';')
display(pf_df)

,data,Hora Medicao,"PRECIPITACAO TOTAL, HORARIO(mm)",RADIACAO GLOBAL(Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)","UMIDADE RELATIVA DO AR, HORARIA(%)","VENTO, VELOCIDADE HORARIA(m/s)"
0,01/01/2018,0,0.0,-3.5,30.3,31.0,0.8
1,01/01/2018,100,0.0,-3.5,30.2,34.0,2.0
2,01/01/2018,200,0.0,-3.5,29.6,47.0,3.2
3,01/01/2018,300,0.0,-3.5,28.1,55.0,4.0
4,01/01/2018,400,0.0,-3.5,27.2,58.0,2.6
...,...,...,...,...,...,...,...
66835,16/08/2025,1900,NaN,NaN,NaN,NaN,NaN
66836,16/08/2025,2000,NaN,NaN,NaN,NaN,NaN
66837,16/08/2025,2100,NaN,NaN,NaN,NaN,NaN
66838,16/08/2025,2200,NaN,NaN,NaN,NaN,NaN


In [11]:
pf_df.fillna(method='ffill', inplace=True)
print(pf_df.isnull().sum())

data                                           0
Hora Medicao                                   0
PRECIPITACAO TOTAL, HORARIO(mm)                0
RADIACAO GLOBAL(Kj/m²)                         0
TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)    0
UMIDADE RELATIVA DO AR, HORARIA(%)             0
VENTO, VELOCIDADE HORARIA(m/s)                 0
dtype: int64


C:\Users\marco\AppData\Local\Temp\ipykernel_32472\1257529808.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pf_df.fillna(method='ffill', inplace=True)


# Tratando dados da variação do volume da barragem

In [12]:
df_barragem = pd.read_csv("dados_tratados_ana_carnaubal.csv")
df_barragem

,data,volume
0,08-16-2021,49.95
1,08-17-2021,49.81
2,08-18-2021,49.68
3,08-19-2021,49.68
4,08-20-2021,49.54
...,...,...
1457,08-12-2025,12.99
1458,08-13-2025,12.99
1459,08-14-2025,12.95
1460,08-15-2025,NaN


In [13]:
df_barragem.fillna(method='ffill', inplace=True)
print(df_barragem.isnull().sum())

data      0
volume    0
dtype: int64


C:\Users\marco\AppData\Local\Temp\ipykernel_32472\3135038113.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_barragem.fillna(method='ffill', inplace=True)


# Unificação dos dois DF


In [14]:
df_volume = df_barragem.copy()
df_volume['data'] = pd.to_datetime(df_volume['data'])
df_clima = pf_df.copy()
df_clima['data'] = pd.to_datetime(df_clima['data'], errors='coerce')

df_final = pd.merge(df_volume,df_clima, on='data', how='inner')
df_final = df_final.sort_values(by='data').reset_index(drop=True)
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13056 entries, 0 to 13055
Data columns (total 8 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   data                                         13056 non-null  datetime64[ns]
 1   volume                                       13056 non-null  float64       
 2   Hora Medicao                                 13056 non-null  int64         
 3   PRECIPITACAO TOTAL, HORARIO(mm)              13056 non-null  float64       
 4   RADIACAO GLOBAL(Kj/m²)                       13056 non-null  float64       
 5   TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)  13056 non-null  float64       
 6   UMIDADE RELATIVA DO AR, HORARIA(%)           13056 non-null  float64       
 7   VENTO, VELOCIDADE HORARIA(m/s)               13056 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 816.1 KB
None


In [15]:
df_final = df_final.sort_values(by='data').reset_index(drop=True)
df_final['variacao_volume'] = df_final['volume'].diff()
df_final.dropna(subset=['variacao_volume'], inplace=True)

caminho = 'dados_tratados_ana_carnaubal_completo.csv'
df_final.to_csv(caminho, index=False, sep=';')

In [16]:
df_final.fillna(method='ffill', inplace=True)
print(df_final.isnull().sum())

data                                           0
volume                                         0
Hora Medicao                                   0
PRECIPITACAO TOTAL, HORARIO(mm)                0
RADIACAO GLOBAL(Kj/m²)                         0
TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)    0
UMIDADE RELATIVA DO AR, HORARIA(%)             0
VENTO, VELOCIDADE HORARIA(m/s)                 0
variacao_volume                                0
dtype: int64


C:\Users\marco\AppData\Local\Temp\ipykernel_32472\1441387021.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final.fillna(method='ffill', inplace=True)


In [17]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
Index: 13055 entries, 1 to 13055
Data columns (total 9 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   data                                         13055 non-null  datetime64[ns]
 1   volume                                       13055 non-null  float64       
 2   Hora Medicao                                 13055 non-null  int64         
 3   PRECIPITACAO TOTAL, HORARIO(mm)              13055 non-null  float64       
 4   RADIACAO GLOBAL(Kj/m²)                       13055 non-null  float64       
 5   TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)  13055 non-null  float64       
 6   UMIDADE RELATIVA DO AR, HORARIA(%)           13055 non-null  float64       
 7   VENTO, VELOCIDADE HORARIA(m/s)               13055 non-null  float64       
 8   variacao_volume                              13055 non-null  float64       
dtypes